In [1]:
install.packages("varhandle")
library(varhandle)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


Activity 표준화 해제

In [2]:
####데이터 불러오기
act = read.csv("train_activity.csv")
tact = read.csv("test_activity.csv")
totalact =  rbind(act, tact)

Warning message in file(file, "rt"):
“cannot open file 'train_activity.csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
####표준화 해제
unstd_totalact = totalact
unstd_totalact[, c(4:38)] = NA

for(i in 4:38)
{
  # 유니크한 벡터 만들기
  uniq     = totalact[, i] %>% unique() %>% sort(decreasing = TRUE)
  
  # 연속한 값들의 차이 구하고 그 차이 값으로 가장 많이 나온 값을 하나의 단위로써 생각하기
  diff     = -diff(uniq) %>% table() %>% as.data.frame() %>% arrange(desc(Freq))
  scale    = diff[1,1] %>% unfactor() %>% as.numeric()
  
  # 위에서 구한 단위로 diff 벡터 및 uniq 벡터 나눠서 scaling
  scaled   = totalact[, i]/scale
  
  # 소숫점 아래 Calibrating
  floating    = scaled - round(scaled)
  floating    = floating %>% table() %>% as.data.frame() %>% arrange(desc(Freq))
  floating    = floating[1,1] %>% unfactor() %>% as.numeric()
  calibrated  = scaled - floating
  
  # Shifting
  shifted = calibrated - min(calibrated)
  
  # 결과값 대입
  unstd_totalact[, i] = shifted
}

Trade 표준화 해제

In [ ]:
#####데이터 불러오기
trade      = read.csv("train_trade.csv")
ttrade     = read.csv("test_trade.csv")
totaltrade = rbind(trade,ttrade)


#####"item_amount" column 표준화 해제
unstd_totaltrade = totaltrade
unstd_totaltrade[, 7] = NA

# 유니크한 벡터 만들기
uniq     = totaltrade[, 7] %>% unique() %>% sort(decreasing = TRUE)

# 연속한 값들의 차이 구하고 그 차이 값으로 가장 많이 나온 값을 하나의 단위로써 생각하기
diff     = -diff(uniq) %>% table() %>% as.data.frame() %>% arrange(desc(Freq))
scale    = diff[1,1] %>% unfactor() %>% as.numeric()

# 위에서 구한 단위로 diff 벡터 및 uniq 벡터 나눠서 scaling
scaled   = totaltrade[, 7]/scale

# 소숫점 아래 Calibrating
floating    = scaled - round(scaled)
floating    = floating %>% table() %>% as.data.frame() %>% arrange(desc(Freq))
floating    = floating[1,1] %>% unfactor() %>% as.numeric()
calibrated  = scaled - floating

# Shifting
shifted = calibrated - min(calibrated)

# 결과값 대입
unstd_totaltrade[, 7] = shifted



#####
# label이 존재하는 레코드 뽑기
from_trade   = unstd_totaltrade[unstd_totaltrade$source_acc_id %in% totalact$acc_id, ]
to_trade     = unstd_totaltrade[unstd_totaltrade$target_acc_id %in% totalact$acc_id, ]


#####
# (id, week, item_amount)의 데이터 프레임 생성
temp_from_trade = from_trade %>% group_by(source_acc_id, trade_week, item_type) %>% summarize(sum = sum(item_amount))
from_trade      = temp_from_trade %>% spread(item_type, sum)
from_trade      = from_trade[order(from_trade$source_acc_id),]
names(from_trade)[c(1,3:8)] = c("acc_id","from_accessory", "from_costume", "from_gem", "from_grocery" ,"from_money", "from_weapon")


temp_to_trade = to_trade %>% group_by(target_acc_id, trade_week, item_type) %>% summarize(sum = sum(item_amount)) 
to_trade      = temp_to_trade %>% spread(item_type, sum)
to_trade      = to_trade[order(to_trade$target_acc_id),]
names(to_trade)[c(1,3:8)] = c("acc_id","to_accessory", "to_costume", "to_gem" ,"to_grocery", "to_money", "to_weapon")

#####
# 데이터 칼럼으로 넓어지게끔 스팬
spanned_trade = merge(from_trade, to_trade, by = c("acc_id", "trade_week"), all = TRUE)
spanned_trade[is.na(spanned_trade)] = 0